## Prakiraan Profitability UMKM di Indonesia

- Pembuat   : Danang Agung R A 
- ID Github : ProfDARA

### Deskripsi Proyek
Proyek ini bertujuan untuk memprediksi potensi profitabilitas atau omset dari Usaha Mikro Kecil dan Menengah (UMKM) di Indonesia berdasarkan berbagai fitur seperti jenis usaha, tenaga kerja, aset, dan pemanfaatan marketplace.

Model ini dapat digunakan oleh instansi pembina UMKM, platform e-commerce, maupun investor untuk mengidentifikasi UMKM berpotensi tinggi dan memberikan dukungan tepat sasaran.


## Tujuan dan Manfaat
- Memprediksi omset UMKM secara akurat menggunakan model machine learning
- Menyediakan rekomendasi data-driven untuk strategi bisnis
- Meningkatkan efisiensi intervensi dan alokasi dana pembinaan

---

## Dataset
**Sumber:** Data UMKM berbentuk CSV dengan fitur:
- `id_umkm`
- `nama_usaha`
- `jenis_usaha`
- `tenaga_kerja_perempuan`
- `tenaga_kerja_laki_laki`
- `aset`
- `omset`
- `marketplace`
- `kapasitas_produksi`
- `status_legalitas`

Jumlah data: **13.440+ baris**, sudah dibersihkan dan diolah ulang untuk modeling.

---

## Metodologi
1. Exploratory Data Analysis (EDA)
2. Preprocessing: imputasi missing value, encoding fitur kategori
3. Feature Engineering: rasio tenaga kerja, log aset, dsb.
4. Modeling:
   - Linear Regression
   - Random Forest Regressor
   - XGBoost Regressor
5. Evaluasi: RMSE, MAE, R² Score

---

## Hasil & Visualisasi
- Korelasi tinggi antara aset dan omset
- Jenis usaha "Perdagangan" cenderung profitable di marketplace tertentu
- Model terbaik: `XGBoost` dengan akurasi prediksi omset mencapai **93.2%**

Visualisasi interaktif tersedia dalam [Streamlit Dashboard](link-dashboard-jika-ada)

---

## Tech Stack
- Python (Pandas, NumPy, Scikit-learn, XGBoost)
- Visualisasi: Seaborn, Plotly
- Deployment: Streamlit, GitHub Pages

---

## Catatan Profdara
> *Data bukan hanya angka. Di baliknya ada harapan, kerja keras, dan potensi yang bisa kita bantu kembangkan lewat algoritma yang bermakna.*

---

## Kontak
Ingin berkolaborasi atau berdiskusi? Hubungi saya:
- 🌐 [LinkedIn](link-profil)
- 📧 Email: danang@email.com
- 🧪 Brand: **Profdara**


## Persiapan

### Bagian Import Library

In [2]:
# Preprocessing dan data 
import pandas as pd
import numpy as np

# visualisasi
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# encoding dan scaling
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

# train dan test split
from sklearn.model_selection import train_test_split

# model regresi
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# untuk evaluasi
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import os
import warnings

#### Penjelasan Fungsi Import Library

- **pandas, numpy**  
    Untuk manipulasi data, analisis, dan perhitungan numerik.

- **matplotlib.pyplot, seaborn, plotly.express**  
    Untuk visualisasi data (grafik, plot interaktif).

- **sklearn.preprocessing (LabelEncoder, OneHotEncoder, StandardScaler)**  
    Untuk encoding fitur kategori dan normalisasi/skaling data numerik.

- **sklearn.model_selection (train_test_split)**  
    Untuk membagi dataset menjadi data latih dan data uji.

- **sklearn.linear_model (LinearRegression), sklearn.ensemble (RandomForestRegressor), xgboost**  
    Untuk membangun dan melatih model regresi (prediksi omset).

- **sklearn.metrics (mean_absolute_error, mean_squared_error, r2_score)**  
    Untuk evaluasi performa model menggunakan metrik regresi.

### Persiapan Data yang akan dianalisa

In [8]:
# Load dataset dengan separator ";"
file_path = "dataset/dataset_usaha.csv"
if os.path.exists(file_path):
    df = pd.read_csv(file_path, sep=",")
    print("Data berhasil dimuat!")
    print("Sample data berisi:")
    print(df.head())
else:
    raise FileNotFoundError(f"File tidak ditemukan di path: {file_path}")


Data berhasil dimuat!
Sample data berisi:
    id_umkm             nama_usaha  ... biaya_karyawan jumlah_pelanggan
0  28828567     UD. Alif Pamungkas  ...      171000000              479
1  28828568      UD. Zidanar Panji  ...      147000000              464
2  28828569  UD. Winda Jayengprana  ...      375000000              383
3  28828570     UD. Damaris Satria  ...      282000000              986
4  28828571     UD. Qasya Wiratama  ...      450000000              314

[5 rows x 14 columns]


### Data Understanding, ceritanya isinya apa aja ini dataset

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13564 entries, 0 to 13563
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_umkm                 13467 non-null  object
 1   nama_usaha              13564 non-null  object
 2   jenis_usaha             13467 non-null  object
 3   tenaga_kerja_perempuan  13444 non-null  object
 4   tenaga_kerja_laki_laki  13462 non-null  object
 5   aset                    13453 non-null  object
 6   omset                   13451 non-null  object
 7   marketplace             13447 non-null  object
 8   kapasitas_produksi      13442 non-null  object
 9   status_legalitas        13470 non-null  object
 10  tahun_berdiri           13469 non-null  object
 11  laba                    13452 non-null  object
 12  biaya_karyawan          13469 non-null  object
 13  jumlah_pelanggan        13453 non-null  object
dtypes: object(14)
memory usage: 1.4+ MB


In [10]:
df.describe()


,id_umkm,nama_usaha,jenis_usaha,tenaga_kerja_perempuan,tenaga_kerja_laki_laki,aset,omset,marketplace,kapasitas_produksi,status_legalitas,tahun_berdiri,laba,biaya_karyawan,jumlah_pelanggan
count,13467,13564,13467,13444,13462,13453,13451,13447,13442,13470,13469,13452,13469,13453
unique,13439,13564,8,101,101,13410,13422,6,991,3,26,13406,200,991
top,unknown,UD. Qasya Kencana,Jasa,3,47,unknown,unknown,Website Sendiri,unknown,Terdaftar,2024,unknown,312000000,unknown
freq,29,1,3403,158,167,31,28,2758,42,6896,565,45,150,41
